In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("audit_5%.csv")

In [3]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,...,NaN,0.0,0.2,0.0,2.4,8.574,0.4,0.5,NaN,1.0
1,NaN,0.00,NaN,0.000,4.83,0.2,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,NaN,10.80,0.6,6.480,NaN,6.0,0.6,...,0.4,0.0,0.2,NaN,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,NaN,0.2,...,0.4,NaN,NaN,0.0,2.0,1.416,0.4,0.5,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,NaN,0.49,0.2,0.098,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.578,0.4,NaN,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,0.2,0.074,0.84,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.3136,0.0
773,55.57,0.24,0.2,NaN,0.04,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,0.2,0.040,0.00,0.2,0.000,0.20,NaN,0.2,...,0.4,0.0,0.2,0.0,2.0,1.440,0.4,0.5,0.2880,0.0


In [4]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk              float64
dtype: object

In [5]:
df.isnull().sum()

Sector_score      56
PARA_A            46
Score_A           38
Risk_A            46
PARA_B            43
Score_B           39
Risk_B            46
TOTAL             34
numbers           38
Score_B.1         26
Risk_C            41
Money_Value       43
Score_MV          35
Risk_D            35
District_Loss     24
PROB              45
RiSk_E            48
History           48
Prob              43
Risk_F            40
Score             37
Inherent_Risk     33
CONTROL_RISK      35
Detection_Risk    42
Audit_Risk        49
Risk               0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 20176
nilai hilang : 1010
persentase : 5.005947660586836


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score      7.216495
PARA_A            5.927835
Score_A           4.896907
Risk_A            5.927835
PARA_B            5.541237
Score_B           5.025773
Risk_B            5.927835
TOTAL             4.381443
numbers           4.896907
Score_B.1         3.350515
Risk_C            5.283505
Money_Value       5.541237
Score_MV          4.510309
Risk_D            4.510309
District_Loss     3.092784
PROB              5.798969
RiSk_E            6.185567
History           6.185567
Prob              5.541237
Risk_F            5.154639
Score             4.768041
Inherent_Risk     4.252577
CONTROL_RISK      4.510309
Detection_Risk    5.412371
Audit_Risk        6.314433
Risk              0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        x1 = np.nan_to_num(x1)  # Mengganti nilai NaN dengan 0
        x2 = np.nan_to_num(x2)  # Mengganti nilai NaN dengan 0
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",dist)

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,5)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [11]:
df2 = df.copy()
dummy = df2.fillna(0)
dummy.fillna(0, inplace=True)
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,...,0.0,0.0,0.2,0.0,2.4,8.574,0.4,0.5,0.0000,1.0
1,0.00,0.00,0.0,0.000,4.83,0.2,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,0.00,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,0.0,0.2,...,0.4,0.0,0.0,0.0,2.0,1.416,0.4,0.5,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,0.00,0.49,0.2,0.098,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.578,0.4,0.0,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,0.2,0.074,0.84,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.3136,0.0
773,55.57,0.24,0.2,0.000,0.04,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,0.2,0.040,0.00,0.2,0.000,0.20,0.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.440,0.4,0.5,0.2880,0.0


In [12]:
euc_dist = {}

In [13]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,5)

Kolom : 1
[3.89, 3.89, 55.57, 55.57, 55.57]
median :  3
min :  3.89
max :  55.57
mean :  34.897999999999996
fuzzy :  [(3.89, 55.57), (55.57, 55.57), (55.57, 55.57), (55.57, 55.57), (3.89, 55.57)]
fuzzy :  55.57
pembilang :  9696.4093
penyebut :  277.85
rata-rata tertimbang :  55.57
 
[2.72, 3.41, 3.41, 3.41, 3.89]
min :  2.72
max :  3.89
mean :  3.368
fuzzy :  [(2.72, 0.0), (3.89, 0.0), (3.41, 0.919540229885057), (3.41, 0.919540229885057), (3.41, 0.919540229885057)]
fuzzy :  None
pembilang :  9.406896551724135
penyebut :  2.758620689655171
rata-rata tertimbang :  3.4100000000000006
 
[2.72, 2.72, 3.41, 3.89, 3.89]
min :  2.72
max :  3.89
mean :  3.3259999999999996
fuzzy :  [(2.72, 0.0), (2.72, 0.0), (3.89, 0.0), (3.41, 0.8510638297872333), (3.89, 0.0)]
fuzzy :  None
pembilang :  2.9021276595744654
penyebut :  0.8510638297872333
rata-rata tertimbang :  3.4099999999999997
 
[1.85, 2.37, 3.89, 21.61, 21.61]
min :  1.85
max :  21.61
mean :  10.266
fuzzy :  [(3.89, 0.2423954372623574), (21.

[0.0, 1.99, 2.72, 2.72, 55.57]
min :  0.0
max :  55.57
mean :  12.6
fuzzy :  [(0.0, 0.0), (55.57, 0.0), (2.72, 0.21587301587301588), (1.99, 0.15793650793650793), (2.72, 0.21587301587301588)]
fuzzy :  None
pembilang :  1.4886428571428572
penyebut :  0.5896825396825397
rata-rata tertimbang :  2.5244818304172276
 
[0.0, 1.85, 1.85, 3.89, 55.57]
min :  0.0
max :  55.57
mean :  12.632000000000001
fuzzy :  [(0.0, 0.0), (1.85, 0.14645345155161493), (55.57, 0.0), (1.85, 0.14645345155161493), (3.89, 0.30794806839772004)]
fuzzy :  None
pembilang :  1.7397957568081064
penyebut :  0.6008549715009499
rata-rata tertimbang :  2.8955335968379448
 
[0.0, 2.72, 3.41, 3.89, 55.57]
min :  0.0
max :  55.57
mean :  13.118
fuzzy :  [(0.0, 0.0), (2.72, 0.20734868120140265), (3.89, 0.29653910657112365), (3.41, 0.2599481628296997), (55.57, 0.0)]
fuzzy :  None
pembilang :  2.6039487726787622
penyebut :  0.763835950602226
rata-rata tertimbang :  3.409041916167665
 
[0.0, 1.85, 1.99, 2.37, 3.89]
min :  0.0
max :  

[0.0, 55.57, 55.57, 55.57, 55.57]
median :  3
min :  0.0
max :  55.57
mean :  44.456
fuzzy :  [(0.0, 55.57), (55.57, 55.57), (55.57, 55.57), (55.57, 55.57), (55.57, 55.57)]
fuzzy :  55.57
pembilang :  12352.0996
penyebut :  277.85
rata-rata tertimbang :  55.57
 
[0.0, 55.57, 55.57, 55.57, 55.57]
median :  3
min :  0.0
max :  55.57
mean :  44.456
fuzzy :  [(0.0, 55.57), (55.57, 55.57), (55.57, 55.57), (55.57, 55.57), (55.57, 55.57)]
fuzzy :  55.57
pembilang :  12352.0996
penyebut :  277.85
rata-rata tertimbang :  55.57
 
Kolom : 2
[0.0, 0.0, 0.0, 0.49, 0.82]
min :  0.0
max :  0.82
mean :  0.262
fuzzy :  [(0.82, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.49, 0.5913978494623656)]
fuzzy :  None
pembilang :  0.28978494623655915
penyebut :  0.5913978494623656
rata-rata tertimbang :  0.49
 
[0.0, 0.45, 1.44, 1.94, 3.14]
min :  0.0
max :  3.14
mean :  1.3940000000000001
fuzzy :  [(3.14, 0.0), (1.94, 0.6872852233676977), (0.45, 0.32281205164992827), (0.0, 0.0), (1.44, 0.9736540664375717)]
fuz

[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)

[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)

...

[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)

[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)

[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)

[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)

[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
median :  3
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)

In [14]:
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.180000,0.6,2.508000,2.50,0.2,0.500,6.680000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.4,8.574,0.4,0.5,1.217396,1.0
1,55.57,0.490000,0.2,0.098000,4.83,0.2,0.966,4.830000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.510800,0.0
2,3.89,0.510000,0.2,0.102000,0.23,0.2,0.046,0.740000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.309600,0.0
3,3.89,1.452128,0.2,0.692703,10.80,0.6,6.480,11.257882,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.506000,1.0
4,3.89,0.554615,0.2,0.206894,0.08,0.2,0.016,0.080000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.416,0.4,0.5,0.283200,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,55.57,0.490000,0.2,0.098000,0.40,0.2,0.080,0.890000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.578,0.4,0.5,0.315600,0.0
772,55.57,0.470000,0.2,0.094000,0.37,0.2,0.074,0.840000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.313600,0.0
773,55.57,0.240000,0.2,0.000000,0.04,0.2,0.008,0.280000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.291200,0.0
774,55.57,0.200000,0.2,0.040000,0.00,0.2,0.000,0.200000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.440,0.4,0.5,0.288000,0.0


In [15]:
dummy.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.0,776.000000,776.000000
mean,19.573106,2.222664,0.352024,1.284536,10.350098,0.312629,6.150256,13.153925,5.065883,0.223196,...,0.490722,0.100515,0.205928,0.052577,2.690351,17.615405,0.570103,0.5,6.868550,0.393041
std,24.046430,4.718041,0.174060,3.418226,49.886781,0.169539,30.023270,51.332721,0.259397,0.079072,...,0.302107,0.528125,0.083262,0.305339,0.846552,54.719498,0.441619,0.0,38.270322,0.488741
min,1.850000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,5.000000,0.200000,...,0.000000,0.000000,0.000000,0.000000,2.000000,1.400000,0.400000,0.5,0.280000,0.000000
25%,2.370000,0.020000,0.200000,0.011500,0.000000,0.200000,0.000000,0.467500,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.000000,1.583500,0.400000,0.5,0.316200,0.000000
50%,3.410000,0.800000,0.200000,0.160000,0.290000,0.200000,0.058000,1.325000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.321609,2.200000,0.400000,0.5,0.511200,0.000000
75%,55.570000,2.422500,0.600000,1.351500,2.970000,0.400000,1.317000,7.527500,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,3.200000,10.663500,0.400000,0.5,3.089578,1.000000
max,59.850000,85.000000,0.600000,51.000000,1264.630000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,2.400000,9.000000,0.600000,5.400000,5.200000,801.262000,5.800000,0.5,961.514400,1.000000


# Klasifikasi

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [17]:
x = dummy.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [35]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[120   4]
 [  1  69]]
0.9742268041237113
